In [ ]:
%pip install opencv-python
%pip install tensorflow

In [ ]:
import cv2
import numpy as np
import time
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="fire_detection.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set up the webcam stream.
cap = cv2.VideoCapture(0)

# Set up the text display parameters.
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (0, 255, 0)
thickness = 2

# Initialize the frame counter and FPS calculation.
frame_count = 0
start_time = time.time()

# Set up the video recording parameters.
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None
recording = False
recording_start_time = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Preprocess the input image.
    resized = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    normalized = resized.astype('float32') / 255.0
    input_data = np.expand_dims(normalized, axis=0)
    
    # Perform the inference.
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Get the predicted class and confidence.
    prediction = output_data[0][0]
    if prediction < 0.5:
        label = 'Fire'
        if frame_count == 0:
            recording = True
            recording_start_time = time.time()
    else:
        label = 'No fire'
        if recording:
            recording = False
            if out is not None:
                out.release()
                out = None
    
    # Display the prediction and frame rate on the screen.
    cv2.putText(frame, label, org, font, fontScale, color, thickness, cv2.LINE_AA)
    frame_count += 1
    if frame_count % 10 == 0:
        end_time = time.time()
        fps = int(frame_count / (end_time - start_time))
        print('FPS:', fps)
        accuracy = round(prediction * 100, 2)
        print('Accuracy:', accuracy, '%')
        frame_count = 0
        start_time = end_time
    
    # Save the frames if fire is detected.
    if recording:
        if out is None:
            out = cv2.VideoWriter('video.mp4', fourcc, fps, (frame.shape[1], frame.shape[0]))
        out.write(frame)
        elapsed_time = time.time() - recording_start_time
        if elapsed_time > 10 or out.get(cv2.CAP_PROP_POS_FRAMES) * out.get(cv2.CAP_PROP_FPS) * 0.000001 > 2:
            recording = False
            out.release()
            out = None
    
    # Display the resulting frame.
    cv2.imshow('Fire Detection',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()